In [13]:
# Install necessary libraries
# !pip install boto3 sagemaker pandas scikit-learn

# Import necessary libraries
import boto3
import sagemaker
import pandas as pd
from sagemaker.predictor import Predictor
from sagemaker.serializers import CSVSerializer
from sagemaker.deserializers import StringDeserializer

# Define variables
endpoint_name = 'justin-linear-regression-endpoint'  # Replace with your SageMaker endpoint name

# Initialize the SageMaker session
sagemaker_session = sagemaker.Session()

# Create a predictor for the endpoint
predictor = Predictor(endpoint_name=endpoint_name,
                      sagemaker_session=sagemaker_session,
                      serializer=CSVSerializer(),
                      deserializer=StringDeserializer())

# Generate some sample test data
sample_data = {
    'MedInc': [8.3252, 8.3014],
    'HouseAge': [41.0, 21.0],
    'AveRooms': [6.9841, 6.2381],
    'AveBedrms': [1.0238, 0.9719],
    'Population': [322.0, 2401.0],
    'AveOccup': [2.5556, 2.1098],
    'Latitude': [37.88, 37.86],
    'Longitude': [-122.23, -122.22]
}

# Convert sample data to DataFrame
sample_df = pd.DataFrame(sample_data)

# # Convert DataFrame to CSV for prediction
# csv_data = sample_df.to_csv(index=False, header=False)

# # Perform inference
# response = predictor.predict(csv_data)

# # Parse the CSV response
# predictions = response.split(',')

# # Ensure the number of predictions matches the number of rows in the DataFrame
# if len(predictions) != len(sample_df):
#     raise ValueError("The number of predictions does not match the number of input rows.")

# # Add predictions to the sample DataFrame
# sample_df['Prediction'] = predictions

# # Save the results to a CSV file
# output_csv = 'inference_results.csv'
# sample_df.to_csv(output_csv, index=False)

# print(f"Inference results saved to {output_csv}")



## 
# Save as compressed json as well 


sagemaker.config INFO - Not applying SDK defaults from location: C:\ProgramData\sagemaker\sagemaker\config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: C:\Users\Admin\AppData\Local\sagemaker\sagemaker\config.yaml


In [16]:
# Install necessary libraries
# !pip install boto3 sagemaker pandas scikit-learn

# Import necessary libraries
import boto3
import sagemaker
import pandas as pd
import numpy as np
from sagemaker.predictor import Predictor
from sagemaker.serializers import CSVSerializer
from sagemaker.deserializers import CSVDeserializer
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Define S3 bucket and SageMaker endpoint
s3_bucket = 'your-s3-bucket'  # Replace with your S3 bucket name
endpoint_name = 'justin-linear-regression-endpoint'  # Replace with your SageMaker endpoint name

# Load and prepare your test data
california = fetch_california_housing()
X, y = california.data, california.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the predictor
predictor = Predictor(
    endpoint_name=endpoint_name,
    sagemaker_session=sagemaker.Session(),
    serializer=CSVSerializer(),
    deserializer=CSVDeserializer()
)

# Preprocess test data
X_test_df = pd.DataFrame(X_test, columns=california.feature_names)

# Test with smaller batches of data
batch_size = 10  # Adjust this value as needed
num_batches = len(X_test_df) // batch_size

all_predictions = []

for i in range(num_batches):
    batch = X_test_df.iloc[i*batch_size:(i+1)*batch_size]
    # Convert batch data to CSV format
    batch_csv = batch.to_csv(header=False, index=False).encode('utf-8')
    # Make predictions using the deployed SageMaker endpoint
    predictions = predictor.predict(batch_csv).decode('utf-8').splitlines()
    predictions = [float(prediction) for prediction in predictions]
    all_predictions.extend(predictions)

# Handle any remaining data not included in the full batches
if len(X_test_df) % batch_size != 0:
    batch = X_test_df.iloc[num_batches*batch_size:]
    batch_csv = batch.to_csv(header=False, index=False).encode('utf-8')
    predictions = predictor.predict(batch_csv).decode('utf-8').splitlines()
    predictions = [float(prediction) for prediction in predictions]
    all_predictions.extend(predictions)

# Convert predictions to a numpy array
all_predictions = np.array(all_predictions)

# Evaluate the model
mse = mean_squared_error(y_test, all_predictions)
rmse = np.sqrt(mse)

print(f'Root Mean Squared Error (RMSE) on test data: {rmse:.4f}')


sagemaker.config INFO - Not applying SDK defaults from location: C:\ProgramData\sagemaker\sagemaker\config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: C:\Users\Admin\AppData\Local\sagemaker\sagemaker\config.yaml


ModelError: An error occurred (ModelError) when calling the InvokeEndpoint operation: Received server error (500) from primary with message "<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 3.2 Final//EN">
<title>500 Internal Server Error</title>
<h1>Internal Server Error</h1>
<p>The server encountered an internal error and was unable to complete your request. Either the server is overloaded or there is an error in the application.</p>
". See https://us-east-1.console.aws.amazon.com/cloudwatch/home?region=us-east-1#logEventViewer:group=/aws/sagemaker/Endpoints/justin-linear-regression-endpoint in account 728939754004 for more information.